### Dependencies

In [2]:
import cv2 as cv
import numpy as np

In [4]:
WIN_NAME = "img2gray"
BG2GRAY = cv.COLOR_BGR2GRAY

def im2gray(path: str) -> None:
    try:
        src = cv.imread(path)
        
        if src is None:  
            raise FileNotFoundError(f"Image not found at path: {path}")
        
        img = cv.cvtColor(src, BG2GRAY)
        cv.imshow(WIN_NAME, img)
        cv.waitKey(0)
        cv.imwrite(f'{path}-2gray.png', img)
    except FileNotFoundError as e:
        print(f"Something went wrong: {e}")

In [5]:
SRC = "ford.png"

im2gray(SRC)

In [ ]:
DEFAULT_CAM = 0

def getCam(cam: int = DEFAULT_CAM) -> None:
    cam = cv.VideoCapture(cam)

    while cam.isOpened():
        _, frame = cam.read()
        cv.imshow('Camera', frame)

        if cv.waitKey(1) == ord('q'):
            break

    cam.release()
    cv.destroyAllWindows()